Math
===

## 00 Find Numbers of Sole Prime Factors
### Description
From integers `n` to `m`, count the numbers that have sole prime factors $x_p$.

The original question I saw this in had specific numbers for $x_p$, but it makes more sense to me to generalize it. The original question was part of a timed test, and I freak out with timed tests and couldn't answer it, which is the only reason I include this problem here.

### Solution


Once I thought about the original problem for a few minutes after the timer expired, I came up with something similar to what is below.

In [29]:
int min = 200, max = 405, count = 0;

for (int i = min; i <= max; i++)
{
    int n = i;

    while (n % 3 == 0)
    {
        n /= 3;
    }

    if (n == 1)
    {
        count++;
        continue;
    }

    while (n % 5 == 0)
    {
        n /= 5;
    }

    if (n == 1)
    {
        count++;
    }
}

count

4

I tried to generalize the problem and then realized this would be a great problem for F#. I felt I was over complicating the problem, so I asked Bing Chat about this part of the subproblem, and then had to modify what it gave me a bit which resulted in the below function.

In [47]:
let isSolelyDivisibleByList (n:int) (list:int list) =
    let mutable x = n
    for factor in list do
        while x % factor = 0 do
            x <- x / factor
    x = 1

[15; 60] |> Seq.map (fun n -> isSolelyDivisibleByList n [3; 5]) |> Array.ofSeq

[ True, False ]

Now we have our generic test function, which we can bind to $x_p$, and then we can replace our `while` loop with a `fold` to count elements that satisfy the function.

In [54]:
let min = 200
let max = 405

let isSolelyDivisibleByListWeCareAbout n = [3;5] |> isSolelyDivisibleByList n

seq { 200 .. 2147483647 } |> Seq.fold (fun acc listItem ->
    if isSolelyDivisibleByListWeCareAbout listItem then acc + 1 else acc) 0

135